## Semantic segmentation model

In [1]:
# Necessary imports for the execution of the code.
# Make sure you have previously executed requirements.txt

'''
# WARNING:

Check https://pytorch.org/get-started/previous-versions/ and install the proper
pytorch and torchvision versions according to your cuda version.

You can figure out your cuda versions with:
/usr/local/cuda/bin/nvcc --version

'''

import os
cwd = os.getcwd()
print(cwd)
import sys
import cv2
import torch
import click
import pickle
import datahandler
import numpy as np
import pandas as pd
import sklearn.metrics
import ETL_lib as Tox
import matplotlib.pyplot as plt

from pathlib import Path
from torch.utils import data
from trainer import train_model, train_clasif_model
from model import createDeepLabv3_resnet_50, binary_fenotypes_wideresnet50
from graphics import plots, Mca, doseresponse
print('started')

/Users/sylviadyballa/repos/zeclinics_app/Terato_AlexFerrando/Code
started


# Model usage

Once we have trained the models, we can proceed by doing predictions over new plates.

**Step 2 (Plate Predictions)**: Once we have our mask extraction and our boolean phenotypes models trained, we can now use them to predict over a plate. 

The following chunk of code will define the plate we want to analyze and will call the main function generate_and_save_predictions from the Tox library. This function saves the roi of each mask inside each corresponding Well folder and generats an overall stats.csv file for the entire plate, which has the predictions on the boolean phenotypes for each Well, among other data

The input parameters to execute the predictions are:

1. *plate_path*: Path to the plate to predict.
2. *batch_size*: Size of the number of images to pass through the nets at the same time. 4 should be a good one.
3. *model_path_seg*: Path to the semantic segmentation model  (Ends with .pt). 
4. *model_path_bools*: Path to the boolean phenotypes model  (Ends with .pt). 
5. *pheno_names*: Name of the different predicted phenotypes.
6. *path_dataframe*: Path to the csv dataframe where data about each well such as the boolean phenotypes is going to be.
7. *masks_names*: It expects a dictionary of key=str and value=list. Each interior list must contain a set of roi files to create a unified mask that will be segmented afterwards The key name of this list will represent the identifier name of the mask. This means if you want to segment the fish outline dorsal joining that roi file with the rois from the eyes just add the following key -> value: 'outline_dorsal' -> ['fishoutline_dorsal', 'eye_up_dorsal', 'eye_down_dorsal']. Atention!: It is mandatory for the masks names to end with \_lat or \_dor to indicate whether the mask will belong to lateral or dorsal image.

In [5]:
#plate_path = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_GB_CPU_01V1/WP4_Developmental_terato/20210831_GB_CPU_01/20210831_GB_CPU_01_PLATE1/20210831_GB_CS_CPU_01_PLATE1/'
#plate_path = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_GB_CPU_01V1/WP4_Developmental_terato/20210831_GB_CPU_01/20210831_GB_CPU_01_PLATE2_PC/20210831_GB_CS_CPU_01_PLATE2_PC'
#plate_path = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_GB_CPU_01V1/WP4_Developmental_terato/20210831_GB_CPU_01/20210831_GB_CPU_01_PLATE3/20210831_GB_CS_CPU_01_PLATE3'
#plate_path = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_GB_CPU_01V1/WP4_Developmental_terato/20210831_GB_CPU_01/20210831_GB_CPU_01_PLATE4/20210831_GB_CS_CPU_01_PLATE4'

# what have we run so far: 
# all plates in: 
# p = "/Volumes/GoogleDrive/Shared drives/AREA_SCI_2019/ZC_2019_JG_BATTELLE_PART2/DCA_VALIDATION/20210915_BATT2_DCA_DS"

#/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/WP2_magt1/20211019_SARP/20211019_SARP_CS_P1

# test folder 
# plate_paths = ['/Users/sylviadyballa/Documents/BAT_test/20210831_GB_CS_CPU_01_PLATE1']

# plate_paths = ['/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/WP2_magt1/raw_data/20211019_SARP/20211019_SARP_CS_P1']

# plate_paths = ['/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_THANEE/20211026_srrm3_P1', 
#     '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_THANEE/20211026_srrm3_P2', 
#     '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_THANEE/20211026_srrm3_P3'
#     ]

# p = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_KT_TP_01V3/20211120_KTTP'
# plate_paths = [item for item in os.listdir(p) if 'PLATE' in item]
# ['20211120_ACYCLOVIR_CS_PLATE1',
#  '20211120_ABACAVIRsulfate_CS_PLATE1',
#  '20211120_DEAB_CS_PLATE1',
#  '20211120_FLAVIDIRAVIR_CS_PLATE1',
#  '20211120_NIFEDIPINE_CS_PLATE1',
#  '20211120_GANCICLOUIR_CS_PLATE1',
#  '20211121_ACYCLOVIRABA_CS_PLATE2',
#  '20211121_GANCICLOVIRACYCLOVIR_CS_PLATE1']

# # xris
# plate_paths = ['/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/DevTerato/alg2/20211005_SA_RP_VAST_PLATE1',
#     '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/DevTerato/alg2/20211005_SA_RP_VAST_PLATE2',
#     '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/DevTerato/ctps1a/20211005_SA_RP_ctps1a/20211006_SARP_CS_P1',
#     '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/DevTerato/flt4/20211020_SARPflt4_CS_R1/',
#     '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/DevTerato/magt1/20211019_SARP_CS_P1/',
#     '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/DevTerato/ppox/20220125_SARP_ppox_P1',
#     ]

# michele
# p = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2022/ZC_2022_TN_MGC_01/20220309_Dev_Terato/ZC_2022_TN_MGC_01V'
# plates = [item for item in os.listdir(p) if '202' in item]
# plate_paths = [os.path.join(p, item) for item in plates]
# plate_paths = ['/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/WP2_AND_5_ppox/raw data/20220125_second_VAST/20220125_SARP_ppox/20220125_SARP_ppox_P1']

# BAT plates
p = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/VAST_images'
plates = [item for item in os.listdir(p) if '202' in item]
len(plates)


6

In [6]:
# run only plates that have no prediction
plates = [
    '',
    '',
    '',
    '',
    '',
    ]

plate_paths = [os.path.join(p, item) for item in plates]

plate_paths

['/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220226_1111_DS_R3',
 '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220226_1240_DS_R3',
 '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220226_1247_DS_R3',
 '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220226_1323_DS_R3',
 '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/VAST_images/20220226_1656_DS_R3']

In [4]:
# # remove roi files


# # plate_paths &
# # remove roi files
# p = "/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_SA_RP_01v01/WP2_AND_5_ppox/raw data/20211110_SARP_pox/SARP_PPOX/20211110_ppox_CS_P1"

# p = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_KT_TP_01V3/20211120_KTTP'
# for root, dirs, files in os.walk(p, topdown=True): 
#     for name in files: 
#         if name.endswith('.roi'): 
#             pass
#             # print(os.path.join(root, name))
#             print(name)
#             # os.remove(os.path.join(root, name))


# plate_paths = []
# for root, dirs, files in os.walk(p, topdown=True): 
#     for name in dirs: 
#         if '_DS_' in name: 
#             plate_paths.append(os.path.join(root, name))         
# print(len(plate_paths))

In [6]:
############ INPUT ############
# plate_path = '../raw_data/20190918_1975_CS_R3'
batch_size = 4
model_path_seg = './Model_masks/weights.pt'
model_path_bools = './Model_pheno/weights.pt'
pheno_names = ['bodycurvature',
              'yolkedema',
              'necrosis',
              'tailbending',
              'notochorddefects',
              'craniofacialedema',
              'scoliosis',
              'snoutjawdefects']

masks_names = {'outline_lat': ['fishoutline_lateral'],
               'heart_lat': ['heart_lateral'],
               'yolk_lat': ['yolk_lateral'],
               'ov_lat': ['ov_lateral'],
               'eyes_dor': ['eye_up_dorsal', 'eye_down_dorsal'],
               'outline_dor': ['fishoutline_dorsal', 'eye_up_dorsal', 'eye_down_dorsal']}

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('Predictions will be using gpu')
else:
    print('Warning! gpu is not available and the model will be predicting in cpu. This could be extremely slow.')

for p, plate_path in enumerate(plate_paths):
    print('working with: ', os.path.split(plate_path)[1], ', ', p+1, ' of ', len(plate_paths))
    path_dataframe = plate_path + '/stats.csv'

    try: 
        Tox.generate_and_save_predictions(plate_path,
                                    batch_size,
                                    model_path_seg,
                                    model_path_bools,
                                    masks_names.keys(),
                                    pheno_names,
                                    device,
                                    path_dataframe)
    except ValueError: 
        print('df arraly length issue - need to fix at some point....') # todo: 20210923_183175_CS_R1 [6]?
    
    # except: 
    #     traceback.print_exc()
    print('finished')

Warning! gpu is not available and the model will be predicting in cpu. This could be extremely slow.
working with:  20220313_DEMARCONEDELTA_SD_R1 ,  1  of  6


100%|██████████| 96/96 [08:58<00:00,  5.61s/it]


Time predict masks 291.4519553184509
Time area 0.14354586601257324
Time length 0.16818833351135254
Time genererate rois 127.11377835273743
Time predict bools 116.85938549041748
finished
working with:  20220313_DEAB_SD_R1 ,  2  of  6


  0%|          | 0/96 [00:00<?, ?it/s]

df arraly length issue - need to fix at some point....
finished
working with:  20220313_LILIAL_SD_R1 ,  3  of  6


...

Time predict masks 648.040874004364
Time area 0.13069415092468262
Time length 0.1443943977355957
Time genererate rois 106.09274578094482
Time predict bools 99.65375804901123
finished
working with:  20220313_ETHYLSAFRANATE_SD_R1 ,  5  of  6


100%|██████████| 96/96 [09:06<00:00,  5.70s/it]


Time predict masks 285.3434946537018
Time area 0.1496589183807373
Time length 0.1754305362701416
Time genererate rois 133.318767786026
Time predict bools 122.18756771087646
finished
working with:  20220313_ENBE_SD_R1 ,  6  of  6


100%|██████████| 96/96 [08:55<00:00,  5.58s/it]

Time predict masks 289.5539095401764
Time area 0.14255023002624512
Time length 0.19095087051391602
Time genererate rois 123.86630058288574
Time predict bools 119.00103759765625
finished


In [ ]:
############ INPUT ############

# plate_path = '../raw_data/20190918_1975_CS_R3'
batch_size = 4
model_path_seg = './Model_masks/weights.pt'
model_path_bools = './Model_pheno/weights.pt'
pheno_names = ['bodycurvature',
              'yolkedema',
              'necrosis',
              'tailbending',
              'notochorddefects',
              'craniofacialedema',
              'scoliosis',
              'snoutjawdefects']

path_dataframe = plate_path + '/stats.csv'

masks_names = {'outline_lat': ['fishoutline_lateral'],
               'heart_lat': ['heart_lateral'],
               'yolk_lat': ['yolk_lateral'],
               'ov_lat': ['ov_lateral'],
               'eyes_dor': ['eye_up_dorsal', 'eye_down_dorsal'],
               'outline_dor': ['fishoutline_dorsal', 'eye_up_dorsal', 'eye_down_dorsal']}



############ CODE ############

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('Predictions will be using gpu')
else:
    print('Warning! gpu is not available and the model will be predicting in cpu. This could be extremely slow.')

Tox.generate_and_save_predictions(plate_path,
                                  batch_size,
                                  model_path_seg,
                                  model_path_bools,
                                  masks_names.keys(),
                                  pheno_names,
                                  device,
                                  path_dataframe)

**Step 2 (Print predictions)**: In the previous chunk, the predictions for the whole plate are done as well as the extraction of some interesting data. Finally, we can see the predictions for each well. This code only needs as input a dictionary of names of wells and their parts to print the masks.

1. *wells*: Dictionary of wells as keys and parts of the fish to plot the masks as values. Remark that fish parts must be one of ('eye_up_dorsal', 'eye_down_dorsal', 'fishoutline_dorsal', 'fishoutline_lateral', 'heart_lateral', 'yolk_lateral', 'ov_lateral').

In [ ]:
############ INPUT ############

wells = {'Well_A01': ['fishoutline_dorsal', 'eye_down_dorsal', 'eye_up_dorsal', 'yolk_lateral']}



############ CODE ############

# Let's retrieve dictionaries with the predictions of the model
dic_images, dic_feno = Tox.dict_from_xml(plate_path)


dorsal_masks = ['eye_up_dorsal', 'eye_down_dorsal', 'fishoutline_dorsal']
lateral_masks = ['fishoutline_lateral', 'heart_lateral', 'yolk_lateral', 'ov_lateral']
cmaps = {'fishoutline_lateral':'Purples' , 
         'heart_lateral': 'Reds', 
         'yolk_lateral': 'Greens', 
         'ov_lateral': 'Blues', 
         'eye_up_dorsal':'Oranges', 
         'eye_down_dorsal':'Oranges', 
         'fishoutline_dorsal':'Purples'}

for well, parts in wells.items():
    # DORSAL IMAGE TO FIGURE 1
    pred_path = os.path.join(plate_path, well, dic_images[well][0])
    im = cv2.imread(pred_path)  # Read image ()
    plt.figure(1)
    plt.imshow(im)
    
    # LATERAL IMAGE TO FIGURE 2
    pred_path = os.path.join(plate_path, well, dic_images[well][1])
    im = cv2.imread(pred_path)  # Read image ()
    plt.figure(2)
    plt.imshow(im)
    
    for part in parts:
        if part in dorsal_masks: plt.figure(1)
        else: plt.figure(2)
        roi_path = os.path.join(plate_path, well, part + '.roi')
        roi = Tox.read_roi_file(roi_path)[part]
        mask_img = np.zeros(im.shape, np.uint8)
        mask_img = Tox.obtain_mask(mask_img, roi)
        plt.imshow(mask_img, cmaps[part], alpha = 0.4)
    
    print(dic_feno[well])

# Exploratory analysis

Now that we have made all the predictions, we can plot our data in order to see some insights and provide a general view of the results obteined.

Before plotting our data we must convert the predictions stored in the stats.csv file into a dataframe, which will be what we will use to process the data and graph them. Then we only need to call the functions of the graphics library and the graphics will be saved in the input folder

The input parameters that we have to initialize to make the plots are:

1.   *dataframe*: The stats.csv converted to a dataframe
2.   *path_graphics*: the folder where we want to save the graphics



In [ ]:
############ INPUT ############

#Our stats.csv has been saved in the path_dataframe variable
dataframe = pd.read_csv(path_dataframe)
path_graphics = plate_path 


############ CODE ###########

#We just have to call the functions

doseresponse(dataframe, path_graphics)
Mca(dataframe, path_graphics)
plots(dataframe, path_graphics)
